이미 학습에 사용한 이미지는 제외하고,
train (10080-이미 학습에 사용한 이미지 개수)장 중 랜덤으로 100장 뽑아서
모델 예측 -> 오탐한 것이 많다면 s키 눌러서 저장
review/images 폴더에 저장됨. 걔네만 따로 수동 라벨링 진행 후 파인튜닝.
q키 눌러서 루틴 종료

In [5]:
import os
import shutil
import random
import cv2
from ultralytics import YOLO
from utils import GetModelPredictResult, DrawBox

# ──────────────────────────────────────
# 0) 설정
# ──────────────────────────────────────
MODEL_PATH   = "runs/train/Finetuned_428/weights/best.pt"
# MODEL_PATH = "runs/tree_095.pt"
SRC_IMG_DIR  = "tree/train"
TRAIN_IMG    = "tree/Datasets/250630_split/images/train"
VAL_IMG      = "tree/Datasets/250630_split/images/val"
SAVE_DIR     = "review/images/428"
NUM_SAMPLES  = 300
IMG_EXT      = (".jpg", ".png")

# 클래스별 색상 (BGR)
CLASS_COLORS = {
    "tree":   ( 34,139, 34),
    "branch": (139, 69, 19),
    "root":   (128,  64,128),
    "crown":  (  0,165,255),
    "fruit":  (  0,215,255),
    "gnarl":  (199, 21,133),
}

# CLASS_COLORS = {
#     "Tree":   ( 34,139, 34),
#     "branch_yn": (139, 69, 19),
#     "root_yn":   (128,  64,128),
#     "crown_yn":  (  0,165,255),
#     "fruit_yn":  (  0,215,255),
#     "gnarl_yn":  (199, 21,133),
# }

os.makedirs(SAVE_DIR, exist_ok=True)

# ──────────────────────────────────────
# 1) 학습에 사용된 이미지 집합
# ──────────────────────────────────────
used_all = {
    f for img_dir in (TRAIN_IMG, VAL_IMG)
    for f in os.listdir(img_dir)
    if f.lower().endswith(IMG_EXT)
}

# ──────────────────────────────────────
# 2) 리뷰 대상 샘플링
# ──────────────────────────────────────
all_imgs = [
    f for f in os.listdir(SRC_IMG_DIR)
    if f.lower().endswith(IMG_EXT)
    # and f not in used_all   # 학습 이미지 제외 시 주석 해제
]
# random.seed(42) # 진짜 랜덤을 원하면 이 줄 제거
random.shuffle(all_imgs)
sampled = all_imgs[:NUM_SAMPLES]

# ──────────────────────────────────────
# 3) 모델 로드
# ──────────────────────────────────────
model = YOLO(MODEL_PATH)

# ──────────────────────────────────────
# 4) 리뷰 루프
# ──────────────────────────────────────
total = len(sampled)
for idx, img_name in enumerate(sampled, start=1):
    img_path = os.path.join(SRC_IMG_DIR, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    print(f"[{idx}/{total}] 검토 중: {img_name}")

    # 예측 결과 가져오기
    res = GetModelPredictResult(model, img_path)
    
    # 윈도우 세팅
    win = "Review (s=save, q=quit)"
    cv2.namedWindow(win, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(win, 1200, 900)

    # DrawBox 를 사용한 시각화
    visualizedImage = DrawBox(img, res, model.names, CLASS_COLORS)
    cv2.imshow(win, visualizedImage)

    key = cv2.waitKey(0) & 0xFF
    cv2.destroyAllWindows()

    if key == ord("q"):
        break
    elif key == ord("s"):
        # shutil.copy2(img_path, os.path.join(SAVE_DIR, img_name))
        # print(f"Saved: {img_name}")
        save_path = os.path.join(SAVE_DIR, img_name)
        cv2.imwrite(save_path, visualizedImage)
    # space나 다른 키는 자동으로 다음 이미지로 넘어갑니다

print("전체 검토 완료.")


[1/300] 검토 중: 나무_13_남_07401.jpg
[2/300] 검토 중: 나무_12_여_06762.jpg
[3/300] 검토 중: 나무_9_남_05173.jpg
[4/300] 검토 중: 나무_12_남_05313.jpg
[5/300] 검토 중: 나무_7_여_07978.jpg
[6/300] 검토 중: 나무_13_여_03539.jpg
[7/300] 검토 중: 나무_12_여_06495.jpg
[8/300] 검토 중: 나무_11_남_09859.jpg
[9/300] 검토 중: 나무_10_남_10776.jpg
[10/300] 검토 중: 나무_9_남_02008.jpg
[11/300] 검토 중: 나무_13_여_03658.jpg
[12/300] 검토 중: 나무_13_여_05941.jpg
전체 검토 완료.


train 이미지 중 한 장씩 랜덤으로 뽑은 후에
원본/contour/adaptive 전처리한 후에 모델이 예측한 걸 보고
q누르면 종료, s 누르면 Prediction 폴더에 저장, space 누르면 저장 없이 다음 이미지

In [22]:
import os
import random
import cv2
import numpy as np
from ultralytics import YOLO
from processing import preprocess_contour, preprocess_adaptive
from utils import GetModelPredictResult, DrawBox  # utils.py에 정확히 정의된 이름을 사용하세요

# ─────────────────────────────────────────────
# 0) 설정
# ─────────────────────────────────────────────
BASE_DIR      = "runs/train"
MODEL_NAME    = "Finetuned_428"
WEIGHT_REL    = "weights/best.pt"
SRC_DIR       = "tree/train"
SAVE_DIR_BASE = "Prediction"

# 클래스 순서와 색상
class_names  = ["tree","branch","root","crown","fruit","gnarl"]
class_colors = {
    "tree":   (255,   0,   0),
    "branch": (  0, 255,   0),
    "root":   (  0,   0, 255),
    "crown":  (255,   0, 255),
    "fruit":  (  0, 128, 128),
    "gnarl":  (128,   0, 128),
}

model_path = os.path.join(BASE_DIR, MODEL_NAME, WEIGHT_REL)
out_dir    = os.path.join(SAVE_DIR_BASE, MODEL_NAME)
os.makedirs(out_dir, exist_ok=True)

# ─────────────────────────────────────────────
# 1) 모델 로드
# ─────────────────────────────────────────────
model = YOLO(model_path)
print(f">>> Loaded model: {model.model_name}")

# ─────────────────────────────────────────────
# 2) 이미지 리스트
# ─────────────────────────────────────────────
all_imgs = [
    f for f in os.listdir(SRC_DIR)
    if f.lower().endswith((".jpg",".png"))
]
if not all_imgs:
    raise RuntimeError("No images found in " + SRC_DIR)

# ─────────────────────────────────────────────
# 3) 메인 루프
# ─────────────────────────────────────────────
print("Press [s] to save, [SPACE] next, [q] to quit")
while True:
    fn   = random.choice(all_imgs)
    path = os.path.join(SRC_DIR, fn)
    raw  = cv2.imread(path)
    if raw is None:
        continue

    # (원한다면) 전처리 전·후 여러 버전 만들기
    variants = [raw
        , preprocess_contour(raw)
        , preprocess_adaptive(raw)
    ]

    vis_list = []
    for img in variants:
        # 1) 모델 예측
        res = GetModelPredictResult(model, img)

        # 2) DrawBox 한 줄로 박스+라벨 표시
        vis = DrawBox(img, res, class_names, class_colors)
        vis_list.append(vis)

    # 3) 화면에 가로로 붙여서 보여주기
    combined = cv2.hconcat(vis_list)
    win_name = "Prediction (s=save / SPACE=next / q=quit)"
    cv2.namedWindow(win_name, cv2.WINDOW_FREERATIO)
    cv2.resizeWindow(win_name, 1800, 600)
    cv2.imshow(win_name, combined)

    key = cv2.waitKey(0) & 0xFF
    cv2.destroyAllWindows()

    if key in (ord("q"), ord("Q")):
        break
    elif key == ord("s"):
        save_path = os.path.join(out_dir, fn)
        cv2.imwrite(save_path, combined)
        print(f"✔️ Saved to {save_path}")
    # SPACE면 그냥 다음으로

print("✅ Finished review.")


>>> Loaded model: runs/train\Finetuned_428\weights/best.pt
Press [s] to save, [SPACE] next, [q] to quit
✔️ Saved to Prediction\Finetuned_428\나무_13_여_11533.jpg
✔️ Saved to Prediction\Finetuned_428\나무_12_남_03995.jpg
✔️ Saved to Prediction\Finetuned_428\나무_13_남_11715.jpg
✔️ Saved to Prediction\Finetuned_428\나무_10_여_11047.jpg
✔️ Saved to Prediction\Finetuned_428\나무_13_여_01685.jpg
✔️ Saved to Prediction\Finetuned_428\나무_13_남_12792.jpg
✔️ Saved to Prediction\Finetuned_428\나무_11_여_03934.jpg
✅ Finished review.


tree 카테고리에서는, 사람과 같이 정해진 클래스의 개수가 있는 것이 아니라
클래스를 몇개 검출했냐 못했냐는 크게 중요하지 않음.
대신 tree 클래스는 무조건 있어야하므로(size, loc 속성에도 필요함)
tree 클래스를 0개 검출한 이미지를 색출함. (학습한 이미지는 제외)

In [ ]:
import os
import shutil
from tqdm import tqdm
from ultralytics import YOLO
from utils import GetModelPredictResult

# ──────────────────────────────────────
# 설정
# ──────────────────────────────────────
MODEL_PATH    = "runs/train/Finetuned_141/weights/best.pt"
SRC_DIR       = "tree/train"
DST_DIR       = "tree/Prediction/tree_0_detected_141"
CONF_THRESH   = 0.5
IOU_THRESH    = 0.5
IMG_EXTS      = (".jpg", ".png")

# 학습에 사용된 이미지 폴더들
TRAIN_USED_DIRS = [
    "tree/Datasets/250630_split/images/train",
    "tree/Datasets/250630_split/images/val",
]

# ──────────────────────────────────────
# 준비
# ──────────────────────────────────────
model = YOLO(MODEL_PATH)

# names dict 을 뒤집어서 {class_name: class_idx} 맵 생성
name_to_idx = {v: k for k, v in model.names.items()}
tree_cls_idx = name_to_idx["tree"]

# 이미 학습에 쓴 파일명 세트 생성
used_fns = set()
for d in TRAIN_USED_DIRS:
    for fn in os.listdir(d):
        if fn.lower().endswith(IMG_EXTS):
            used_fns.add(fn)

os.makedirs(DST_DIR, exist_ok=True)

imageCount = 0

for fn in tqdm(os.listdir(SRC_DIR), desc="Scanning images"):
    if not fn.lower().endswith(IMG_EXTS):
        continue

    # 학습셋에 이미 포함된 파일은 건너뛰기
    if fn in used_fns:
        continue

    img_path = os.path.join(SRC_DIR, fn)

    res = GetModelPredictResult(model, img_path, 0.5)

    clses = res.boxes.cls.cpu().numpy()
    # tree 클래스가 하나도 없으면 복사
    if not (clses == tree_cls_idx).any():
        shutil.copy2(img_path, os.path.join(DST_DIR, fn))
        imageCount += 1

print(f"✅ tree 클래스를 못잡은 이미지 {imageCount} 장이 복사 완료되었습니다.")


현재 학습시킨 모델이 train 이미지 10080장을
어떻게 예측하는 건지 한 번에 다 돌려보고 저장

In [ ]:
from tqdm import tqdm
import os
import cv2
from ultralytics import YOLO
from utils import GetModelPredictResult

def visualize_predictions(model, img_paths, save_dir="visualizations"):
    """
    Run inference on a list of images and save annotated results to a single directory with progress bar.
    
    Args:
        model:      YOLO model instance
        img_paths:  list of image file paths
        save_dir:   folder to save annotated images
    """
    os.makedirs(save_dir, exist_ok=True)

    for img_path in tqdm(img_paths, desc="Visualizing images"):
        # run prediction
        res = GetModelPredictResult(model, img_path)
        # plot annotated image (all detections)
        annotated = res.plot()  # numpy RGB array
        bgr = cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR)
        fname = os.path.basename(img_path)
        
        # save annotated image to single folder
        cv2.imwrite(os.path.join(save_dir, fname), bgr)
    
    print(f"✅ All visualizations saved to: {save_dir}")

# Example usage:
model = YOLO("runs/train/Finetuned_428/weights/best.pt")
img_dir = "tree/train"
img_paths = [
    os.path.join(img_dir, f) for f in os.listdir(img_dir)
    if f.lower().endswith(('.jpg', '.png'))
]

visualize_predictions(
    model, img_paths,
    save_dir="review/Visualizations_all"
)


Visualizing images: 100%|██████████| 10080/10080 [06:36<00:00, 25.42it/s]

✅ All visualizations saved to: review/Visualizations_all


모델 성능 지표 확인하기

In [ ]:
from ultralytics import YOLO

MODEL_PATH = "runs/train/Finetuned_428/weights/best.pt"
DATA_PATH = "tree/Datasets/250630_split/data.yaml"

model = YOLO(MODEL_PATH)
metrics = model.val(data=DATA_PATH)

print("mAP50-95:", metrics.box.map)       # mAP@0.5-0.95
print("mAP50:", metrics.box.map50)        # mAP@0.5
print("Precision:", metrics.box.mp)      # Mean Precision
print("Recall:", metrics.box.mr)         # Mean Recall

Ultralytics 8.3.159  Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1060.7292.3 MB/s, size: 91.1 KB)


val: Scanning C:\Users\UserK\PyProjects\TeamProject1\tree\Datasets\250630_split\labels\val.cache... 89 images, 0 backgrounds, 0 corrupt: 100%|██████████| 89/89 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.45it/s]


                   all         89        721      0.999      0.999      0.995      0.982
                  Tree         87         87      0.998          1      0.995      0.995
             branch_yn         86        183          1      0.995      0.995      0.973
               root_yn         89         91          1      0.998      0.995      0.976
              crown_yn         87         87      0.998          1      0.995      0.994
              fruit_yn         81        228      0.999          1      0.995      0.979
              gnarl_yn         28         45      0.996          1      0.995      0.977
Speed: 1.3ms preprocess, 15.9ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs\detect\val4
mAP50-95: 0.9824236955990404
mAP50: 0.995
Precision: 0.9985102688802908
Recall: 0.9987327372477971


tree 2개 이상 잡은 이미지 검출 코드 => (작동 안함, 1개만 잡은 이미지도 포함되었음..)

In [18]:
import os
import cv2
from ultralytics import YOLO
from tqdm import tqdm

# ──────────────────────────────────────
# 설정
# ──────────────────────────────────────
MODEL_PATH   = "runs/train/Finetuned_428/weights/best.pt"
IMG_DIR      = "tree/train"
OUT_DIR      = "tree/Prediction/tree_over_detected"

os.makedirs(OUT_DIR, exist_ok=True)

# ──────────────────────────────────────
# 모델 로드
# ──────────────────────────────────────
model = YOLO(MODEL_PATH)
names = model.names

# ──────────────────────────────────────
# 이미지 필터링 및 저장
# ──────────────────────────────────────
for filename in tqdm(os.listdir(IMG_DIR), desc="Processing images"):
    if not filename.lower().endswith((".jpg", ".png")):
        continue
    img_path = os.path.join(IMG_DIR, filename)
    img = cv2.imread(img_path)
    if img is None:
        continue

    # 모델 예측
    result = model.predict(source=img, verbose=False)[0]
    classes = result.boxes.cls.cpu().numpy()

    # 'tree' 클래스 개수 세기
    tree_count = sum(1 for c in classes if names[int(c)] == 'tree')

    # 2개 이상 검출된 이미지 저장
    if tree_count >= 2:
        save_path = os.path.join(OUT_DIR, filename)
        cv2.imwrite(save_path, img)

print(f"Done! Over-detected images saved to {OUT_DIR}")


Processing images: 100%|██████████| 10080/10080 [04:32<00:00, 37.06it/s]

Done! Over-detected images saved to tree/Prediction/tree_over_detected
